In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd
import statsmodels.api as sm

import sys
sys.path.append("../")
from bargains_linear import * 

In [2]:
def gen_data(params,nobs=100,var=5):
    wtp,cost,beta1,beta2,emc1,emc2=params
    emc = emc1,emc2
    passive_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2']) 
    active_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'])
    seq_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'])

    for i in range(0,nobs):
        #sequential
        mc1 = np.random.normal(loc=0, scale=var, size=(2,)) + np.array([emc1,emc2])
        mc2 = np.random.normal(loc=0, scale=1, size=(2,))
        mc = mc1+mc2

        #seq results
        phi1,phi2 = seq_bargain(25,cost,wtp,mc)
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        seq_result = pd.concat( [seq_result, row] )


        #active results
        phi1,phi2 =simult_bargain(25,25, cost,wtp,mc, active=True)
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        active_result = pd.concat( [active_result, row] )


        #passive results
        phi1,phi2 =simult_bargain(25,25, cost,wtp,mc, active=False)
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        passive_result = pd.concat( [passive_result, row] )
    return seq_result, active_result,passive_result




def print_test(data):
    model1 = sm.OLS(data['phi2'],sm.add_constant(data[['phi1','c1']]))
    param, se = model1.fit().params.rename('param'),model1.fit().bse.rename('se')
    return pd.merge(param,se,left_index=True,right_index=True)

seq_result,active_result,passive_result = gen_data([25,5,.5,.5,1,1],nobs=500)
print(print_test(seq_result))
print(print_test(active_result))
print(print_test(passive_result))

print('----------------------')
seq_result,active_result,passive_result = gen_data([25,5,.5,.5,0,0],nobs=500)
print(print_test(seq_result))
print(print_test(active_result))
print(print_test(passive_result))
print('----------------------')


seq_result,active_result,passive_result = gen_data([25,5,.5,.5,3,3],nobs=500)
print(print_test(seq_result))
print(print_test(active_result))
print(print_test(passive_result))
print('----------------------')


           param        se
const  19.147414  0.942454
phi1    0.003597  0.050765
c1     -0.208177  0.073450
           param        se
const  23.405323  0.451754
phi1   -0.765787  0.033023
c1      0.109810  0.015612
          param        se
const  5.863883  0.380471
phi1   0.191212  0.049071
c1     0.207631  0.015277
----------------------
           param        se
const  16.609490  0.896676
phi1    0.163667  0.049670
c1     -0.114136  0.076490
           param        se
const  23.649376  0.477988
phi1   -0.784272  0.035790
c1      0.090594  0.017800
           param        se
const  13.978674  0.372554
phi1   -0.878285  0.049660
c1      0.065181  0.011661
----------------------
           param        se
const  14.683511  0.927182
phi1    0.287272  0.045576
c1     -0.012593  0.060449
           param        se
const  22.335084  0.414577
phi1   -0.702414  0.028929
c1      0.095476  0.013828
          param        se
const  5.586271  0.454270
phi1   0.164970  0.056937
c1     0.222969 

In [3]:
seq_result = pd.read_csv('fake_data/seq_data.csv',index_col=0)
active_result = pd.read_csv('fake_data/active_data.csv',index_col=0)
passive_result = pd.read_csv('fake_data/passive_data.csv',index_col=0)


def run_test(data):
    regression_result = print_test(data)
    test_statistic = regression_result.loc['c1','param']/regression_result.loc['c1','se']
    test_result = (test_statistic > 1.96)
    return test_result

#when null is true:
rejections = 0
for i in range(1000):
    #resample from seq
    subsample = seq_result.sample(frac=0.5)
    rejections = rejections + run_test(subsample)
print('sequential',rejections/1000)

rejections = 0
for i in range(1000):
    #resample from seq
    subsample = active_result.sample(frac=0.5)
    rejections = rejections + run_test(subsample)
print('active',rejections/1000)



rejections = 0
for i in range(1000):
    #resample from seq
    subsample = passive_result.sample(frac=0.5)
    rejections = rejections + run_test(subsample)
print('passive',rejections/1000)


sequential 0.003
active 1.0
passive 0.987
